In [4]:
import re
from typing import Dict, Generator, List, Set

from pyswip import Prolog

In [2]:
prolog = Prolog()
prolog.assertz("father(michael,john)")
prolog.assertz("father(michael,gina)")
list(prolog.query("father(michael,X)"))  # [{'X': 'john'}, {'X': 'gina'}]

[{'X': 'john'}, {'X': 'gina'}]

In [13]:
prolog1.consult("knowledge_base1.pl")
print(list(prolog1.query("father(michael,X)")))
print(list(prolog2.query("father(peter,X)")))
prolog2 = Prolog()
prolog2.consult("knowledge_base2.pl")
print(list(prolog1.query("father(michael,X)")))
print(list(prolog2.query("father(peter,X)")))

[{'X': 'john'}, {'X': 'gina'}]
[]
[]
[{'X': 'john'}, {'X': 'gina'}]
